In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval = -1e-1, maxval = 1e-1)
        self.W = tf.Variable(w_initial_value)
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [3]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size = 128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = np.ceil(len(images) / batch_size).astype(int)

    def next(self):
        images = self.images[self.batch_size * self.index:self.batch_size * (self.index + 1)]
        labels = self.labels[self.batch_size * self.index:self.batch_size * (self.index + 1)]
        self.index += 1
        return images, labels

In [4]:
def update_weights(gradients, weights, learning_rate = 1e-3):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

def update_weights2(gradients, weights, learning_rate = 1e-3):
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    optimizer.apply_gradient(zip(gradients, weights))

def one_step_training(model, image_batch, label_batch):
    with tf.GradientTape() as tape:
        prediction = model(image_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            label_batch, prediction
        )
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)   
    update_weights(gradients, model.weights) 
    return average_loss

def fit(model, images, labels, epochs, batch_size = 128):
    for epoch in range(epochs):
        print(f"epoch: {epoch}")
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_number in range(batch_generator.num_batches):
            image_batch, label_batch = batch_generator.next()
            loss = one_step_training(model, image_batch, label_batch)
            if batch_number % 100 == 0:
                print(f"loss at batch {batch_number}: {loss:.2f}")

In [5]:
model = NaiveSequential([
    NaiveDense(input_size = 28*28, output_size = 512, activation = tf.nn.relu),
    NaiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape(60000, -1).astype("float32") / 255
test_images = test_images.reshape(10000, -1).astype("float32") / 255

fit(model, train_images, train_labels, epochs = 5, batch_size = 128)

predictions = model(test_images)
print(type(predictions))
predictions = predictions.numpy()
predictions = np.argmax(predictions, axis = 1)
matches = predictions == test_labels

print(f"accuracy: {matches.mean():.2f}")

epoch: 0
loss at batch 0: 2.47
loss at batch 100: 2.22
loss at batch 200: 2.00
loss at batch 300: 1.86
loss at batch 400: 1.81
epoch: 1
loss at batch 0: 1.69
loss at batch 100: 1.68
loss at batch 200: 1.49
loss at batch 300: 1.44
loss at batch 400: 1.43
epoch: 2
loss at batch 0: 1.31
loss at batch 100: 1.35
loss at batch 200: 1.16
loss at batch 300: 1.16
loss at batch 400: 1.18
epoch: 3
loss at batch 0: 1.06
loss at batch 100: 1.12
loss at batch 200: 0.95
loss at batch 300: 0.98
loss at batch 400: 1.01
epoch: 4
loss at batch 0: 0.90
loss at batch 100: 0.96
loss at batch 200: 0.80
loss at batch 300: 0.85
loss at batch 400: 0.90
<class 'tensorflow.python.framework.ops.EagerTensor'>
accuracy: 0.83
